# Correlation analysis, CCA and feature importance: Antenatal coverage

## Outcome variables:
- Maternal mortality rate: rate_anc_mortality
- Under 5 mortality rate: rate_under5y_mortality
- Antenatal coverage (ANC): prop_antenatal_coverage
- Proportion of unmet contraceptive need: prop_unmet_need_family_planing
- Oral rehydration solution (ORS): still to caclulate

In [1]:
import re
import collections
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.cross_decomposition import CCA
from sklearn.preprocessing import StandardScaler
%matplotlib inline

In [2]:
def remove_miss_vars(input_df):
    df = input_df.copy(deep=True)
    remove_list = []
    for var in df.columns:
        if any(df[var].isna()):
            remove_list.append(var)
    return df.drop(remove_list, axis=1)

def impute_miss_vars(input_df):
    df = input_df.copy(deep=True)
    for var in df.columns:
        if any(df[var].isna()):
            df[var].fillna(df[var].mean, inplace=True)
    return df

def intersect_dfs(input_df1, input_df2):
    df1 = input_df1.copy(deep=True)
    df2 = input_df2.copy(deep=True)
    subset_var = list(set(list(df1.columns)).intersection(set(list(df2.columns))))
    return df1[subset_var], df2[subset_var]

## STEP 1: Import data and data processing: remove absolute and remove missing values

In [3]:
DATA2011 = '/Users/edinhamzic/Symphony/wb_bangladesh/Bangladesh/output/all/all2011.csv'
DATA2016 = '/Users/edinhamzic/Symphony/wb_bangladesh/Bangladesh/output/all/all2016.csv'
DHIS2_VARS = '/Users/edinhamzic/Symphony/wb_bangladesh/Bangladesh/output/all/DHIS_Rate_Absolute.csv'
OUT = '/Users/edinhamzic/Symphony/wb_bangladesh/Bangladesh/output/all/'

### Removing all absolute variables. Including only indicators available from the DHIS2 system
- Originally there were over 349 variables in the system that we retrieved from the DHIS2 system in collaboration with HMIS officials
- We kept 279 indicators

In [4]:
d2011 = pd.read_csv(DATA2011)
d2016 = pd.read_csv(DATA2016)
dhis2vars = pd.read_csv(DHIS2_VARS)
print(dhis2vars.shape)
tmp = dhis2vars[dhis2vars['Rate_Absolute'] == 'Absolute']
print(tmp.shape)
vars_remove = list(tmp['Full_name'])

(349, 4)
(279, 4)


In [5]:
d2011 = d2011.drop(vars_remove, axis=1)
d2016 = d2016.drop(vars_remove, axis=1)

## Removing all object/string variables/indicators and fill missing values with mean

In [6]:
d2011.shape
d2011 = d2011.set_index(['DistrictName'])
print(d2011.shape)
d2011 = d2011.drop(['DistrictGeo'], axis=1)
print(d2011.shape)
subset_vars = [var for var, var_type in zip(d2011.dtypes.index, d2011.dtypes) if str(var_type) != 'object'] 
d2011 = d2011[subset_vars]
d2011 = d2011.fillna(d2011.mean())
#d2011 = remove_miss_vars(input_df=d2011)
print(d2011.shape)
d2011.head()

(64, 188)
(64, 187)
(64, 187)


,TT1_Mother0-11MChildren,Measles_Children23M,TT4_Mother0-11MChildren,OPV3_Children12M,OPV2_Children23M,OPV3_Children23M,PENTA2_Children12M,TT3_Mother0-11MChildren,PENTA3_Children23M,VitACoverage_Children12-59M,...,prop_pop_rural.1,prop_institutional_delivery,prop_current_contraceptive,prop_female_head,prop_antenatal_care4.,prop_unmet_need_family_planing,prop_pop_women.1,dependency_ratio.1,prop_registered_under5,sex_ratio.1
DistrictName,,,,,,,,,,,,,,,,,,,,,
Bagerhat,90.0,88.1,52.9,93.3,96.0,93.3,96.0,71.4,85.6,96.7,...,11.48,30.66,65.52,11.29,26.56,7.89,44.85,74.37,32.84,93.31
Bandarban,89.0,82.3,54.3,88.8,90.8,89.4,90.2,72.4,84.4,83.3,...,0.00,0.00,45.45,11.29,0.00,27.27,127.03,107.50,31.25,84.44
Barguna,99.0,88.1,47.6,94.7,98.6,95.4,98.6,72.4,87.9,96.7,...,6.48,10.56,72.36,2.97,20.97,11.14,25.33,70.73,41.40,105.44
Barisal,97.1,86.8,52.4,94.5,98.1,94.5,98.1,77.1,86.9,79.0,...,8.22,21.53,64.33,5.10,26.53,13.31,24.48,75.53,38.53,89.02
Bhola,98.6,85.0,67.6,93.0,96.5,93.0,95.8,85.7,86.3,87.1,...,7.40,9.64,68.40,5.65,29.51,10.81,26.74,80.16,21.19,94.76


In [7]:
d2016.shape
d2016 = d2016.set_index(['DistrictName'])
print(d2016.shape)
d2016 = d2016.drop(['DistrictGeo'], axis=1)
print(d2016.shape)
subset_vars = [var for var, var_type in zip(d2016.dtypes.index, d2016.dtypes) if str(var_type) != 'object'] 
d2016 = d2016[subset_vars]
d2016 = d2016.fillna(d2016.mean())
# d2016 = remove_miss_vars(input_df=d2016)
print(d2016.shape)
d2016.head()

(64, 188)
(64, 187)
(64, 187)


,TT1_Mother0-11MChildren,Measles_Children23M,TT4_Mother0-11MChildren,OPV3_Children12M,OPV2_Children23M,OPV3_Children23M,PENTA2_Children12M,TT3_Mother0-11MChildren,PENTA3_Children23M,VitACoverage_Children12-59M,...,prop_pop_rural.1,prop_institutional_delivery,prop_current_contraceptive,prop_female_head,prop_antenatal_care4.,prop_unmet_need_family_planing,prop_pop_women.1,dependency_ratio.1,prop_registered_under5,sex_ratio.1
DistrictName,,,,,,,,,,,,,,,,,,,,,
Bagerhat,98.1,92.2,61.9,90.8,97.1,91.0,96.8,85.3,91.0,82.0,...,14.38,26.49,67.82,4.17,25.13,11.00,39.86,56.82,37.56,94.78
Bandarban,94.8,89.8,79.6,87.8,94.8,87.8,94.8,86.1,87.8,84.1,...,0.00,75.00,63.64,25.49,75.00,13.64,77.52,45.16,40.00,84.93
Barguna,98.8,94.9,64.4,93.0,98.3,93.6,97.9,88.4,93.6,96.8,...,9.48,27.54,73.30,8.06,47.46,8.22,28.64,68.24,25.10,87.82
Barisal,100.0,97.1,79.3,95.5,99.3,96.0,99.3,96.7,96.0,100.0,...,28.48,46.33,64.26,6.99,32.39,10.08,30.97,59.68,23.40,94.74
Bhola,100.0,96.6,79.0,94.5,99.8,94.5,99.8,94.1,94.5,98.4,...,9.99,11.27,67.28,3.53,15.62,10.01,26.58,69.35,17.64,101.71


## Renaming variables

In [8]:
var_names = pd.read_csv("/Users/edinhamzic/Symphony/wb_bangladesh/Bangladesh/output/all/IndicatorsNames_2011_2016.csv")
var_names.head()

if all(d2016.columns == d2011.columns):
    print(True)
    if all([var1==var2 for var1,var2 in zip(list(d2016.columns), list(var_names['Indicators 2011']))]):
        new_names = []
        for key, var in enumerate(d2016.columns):
            new_names.append(var_names.loc[key,'Description'])
        d2011.columns = new_names
        d2016.columns = new_names
d2011 = d2011.drop('DHS: Proportion of antental coverage 4+', axis=1)
d2016 = d2016.drop('DHS: Proportion of antental coverage 4+', axis=1)

True


## STEP 2: Outcome variable: Antenatal coverage

###  Antenatal coverage

In [9]:
print(f"Mean: {round(d2011['DHS: Proportion of antenatal coverage'].mean(),2)}",
      f" Standard deviation: {round(d2011['DHS: Proportion of antenatal coverage'].std(),2)}")

Mean: 37.97  Standard deviation: 13.59


In [10]:
print(f"Mean: {round(d2016['DHS: Proportion of antenatal coverage'].mean(),2)}",
      f" Standard deviation: {round(d2016['DHS: Proportion of antenatal coverage'].std(),2)}")

Mean: 56.92  Standard deviation: 17.35


### Scaled and normalized data


In [11]:
print(d2011.shape)
drop_columns = []
for var in d2011.columns:
    if "index" in var:
        drop_columns.append(var)
drop_columns = list(set(drop_columns))
print(drop_columns)
d2011.drop(drop_columns, inplace=True, axis=1)
print(d2011.shape)

(64, 186)
[]
(64, 186)


In [12]:
print(d2016.shape)
drop_columns = []
for var in d2016.columns:
    if "index" in var:
        drop_columns.append(var)
drop_columns = list(set(drop_columns))
print(drop_columns)
d2016.drop(drop_columns, inplace=True, axis=1)
print(d2016.shape)

(64, 186)
[]
(64, 186)


### Standar scaling of data
- Standardize features by removing the mean and scaling to unit variance

In [13]:
s_data2011 = StandardScaler().fit_transform(d2011)
s_data2011 = pd.DataFrame(s_data2011, columns=d2011.columns)
print(s_data2011.shape)
s_data2011 = remove_miss_vars(input_df=s_data2011)
print(s_data2011.shape)

(64, 186)
(64, 186)


/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/anaconda3/lib/python3.7/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [14]:
d2016[d2016==np.inf]=np.nan
d2016.fillna(d2016.mean(), inplace=True)
s_data2016 = StandardScaler().fit_transform(d2016)
s_data2016 = pd.DataFrame(s_data2016, columns=d2016.columns)
print(s_data2016.shape)
s_data2016 = remove_miss_vars(input_df=s_data2016)
print(s_data2016.shape)


(64, 186)
(64, 186)


/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/anaconda3/lib/python3.7/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


### Performing PCA on both years 2011 and 2016: Preliminary

In [15]:
pca = PCA(.95)
pca2011 = pca.fit(s_data2011)
print(pca.n_components_)

40


In [16]:
pca = PCA(.95)
pca2016 = pca.fit(s_data2016)
print(pca.n_components_)

42


## Performing Canonical Correlation Analysis (CCA) and correlation analysis for 2011
- The aim of this part is to identify variables highly correlated with antental coverage

### Canonical Correlation Analysis (CCA) 2011

In [17]:
cca_anc = CCA(copy=True, max_iter=1000, n_components=40, scale=True, tol=1e-06)
cca_anc.fit(s_data2011.drop('DHS: Proportion of antenatal coverage', axis=1),
                 s_data2011['DHS: Proportion of antenatal coverage'])
print(cca_anc.score(s_data2011.drop('DHS: Proportion of antenatal coverage', axis=1),
                         s_data2011['DHS: Proportion of antenatal coverage']))

0.9993459340496103


/anaconda3/lib/python3.7/site-packages/sklearn/cross_decomposition/pls_.py:291: UserWarning: Y residual constant at iteration 1
  warnings.warn('Y residual constant at iteration %s' % k)


In [18]:
CCA_coeff_anc = pd.DataFrame({'Indicators': list(s_data2011.drop('DHS: Proportion of antenatal coverage', axis=1).columns),
                                   'CCA_coeff': cca_anc.coef_[:,0],
                                   'CCA_coeff_abs': np.absolute(cca_anc.coef_[:,0]),})
CCA_coeff_anc.sort_values(by='CCA_coeff_abs', ascending =False).head()
CCA_coeff_anc = CCA_coeff_anc[CCA_coeff_anc['CCA_coeff_abs'] > 0.1]
display(CCA_coeff_anc.head())
print(f"Number of indicators from the original file with coefficent values above 0.1 is {CCA_coeff_anc.shape[0]}")
CCA_coeff_anc.to_csv(OUT+'/cca_antenatal_coverage_2011.csv', index=False, index_label=False)

,Indicators,CCA_coeff,CCA_coeff_abs
2,Crude coverage TT4 among mothers with 0-11 mon...,-0.182782,0.182782
15,Crude coverage TT5 among mothers with 0-11 chi...,-0.142744,0.142744
21,"Crude coverage BCG children 12 months, %",0.102570,0.102570
25,Proportion of women (15-45 years),-0.122544,0.122544
30,Proportion of registered births,0.124551,0.124551


Number of indicators from the original file with coefficent values above 0.1 is 31


### Correlation analysis 2011

In [19]:
import scipy.stats as stats
all(s_data2011.columns == s_data2016.columns)

True

In [20]:
corr2011 = s_data2011.corr()

In [21]:
antenatal_coverage_2011 = corr2011[['DHS: Proportion of antenatal coverage']]
antenatal_coverage_2011['abs_antenatal_coverage'] = np.absolute(antenatal_coverage_2011['DHS: Proportion of antenatal coverage'])
antenatal_coverage_2011 = antenatal_coverage_2011.sort_values(by='abs_antenatal_coverage', ascending=False)
antenatal_coverage_2011.drop('DHS: Proportion of antenatal coverage',axis=0, inplace=True)
antenatal_coverage_2011.reset_index(inplace=True)
corr_pvalues = []
for var in antenatal_coverage_2011['index']:
    pvalue = stats.pearsonr(s_data2011[var], s_data2011['DHS: Proportion of antenatal coverage'])[1]
    corr_pvalues.append(pvalue)
antenatal_coverage_2011['p_value'] = corr_pvalues
antenatal_coverage_2011 = antenatal_coverage_2011[antenatal_coverage_2011['p_value'] < 0.05]
antenatal_coverage_2011.sort_values(by = 'abs_antenatal_coverage', ascending=False)
display(antenatal_coverage_2011.head())
print(f"Number of indicators from the original file with correlation coefficient values above 0.1 is {antenatal_coverage_2011.shape[0]}")
antenatal_coverage_2011.to_csv(OUT+'/corr_antenatal_coverage_2011.csv', index=False, index_label=False)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/anaconda3/lib/python3.7/site-packages/scipy/stats/stats.py:3010: RuntimeWarning: invalid value encountered in double_scalars
  r = r_num / r_den


,index,DHS: Proportion of antenatal coverage,abs_antenatal_coverage,p_value
0,DHS: Proportion of C-sections,0.684099,0.684099,4.623235e-10
1,DHS: Proportion of attendant deliveries,0.671220,0.671220,1.280299e-09
2,DHS: Proportion of institutional deliveries,0.632610,0.632610,2.048893e-08
3,DHS: Proportion of women (15-45 years),0.543440,0.543440,3.479477e-06
4,DHS: Dependency ratio,-0.517718,0.517718,1.180959e-05


Number of indicators from the original file with correlation coefficient values above 0.1 is 46


### Combine results

In [22]:
antenatal_coverage_vars_2011 = list(set(CCA_coeff_anc['Indicators']).union(set(antenatal_coverage_2011['index'])))
len(list(set(CCA_coeff_anc['Indicators']).union(set(antenatal_coverage_2011['index']))))

69

## Performing Canonical Correlation Analysis (CCA) and correlation analysis for 2016
- The aim of this part is to identify variables highly correlated with antenatal coverage

### Canonical Correlation Analysis (CCA) 2016

In [23]:
cca_anc = CCA(copy=True, max_iter=500, n_components=40, scale=True, tol=1e-06)
cca_anc.fit(s_data2016.drop('DHS: Proportion of antenatal coverage', axis=1),
                 s_data2016['DHS: Proportion of antenatal coverage'])
print(cca_anc.score(s_data2016.drop('DHS: Proportion of antenatal coverage', axis=1),
                         s_data2016['DHS: Proportion of antenatal coverage']))

0.9563434138827416


/anaconda3/lib/python3.7/site-packages/sklearn/cross_decomposition/pls_.py:291: UserWarning: Y residual constant at iteration 1
  warnings.warn('Y residual constant at iteration %s' % k)


In [24]:
CCA_coeff_anc = pd.DataFrame({'Indicators': list(s_data2016.drop('DHS: Proportion of antenatal coverage', axis=1).columns),
                                   'CCA_coeff': cca_anc.coef_[:,0],
                                   'CCA_coeff_abs': np.absolute(cca_anc.coef_[:,0]),})
CCA_coeff_anc.sort_values(by='CCA_coeff_abs', ascending =False).head()
CCA_coeff_anc = CCA_coeff_anc[CCA_coeff_anc['CCA_coeff_abs'] > 0.1]
display(CCA_coeff_anc.head())
print(f"Number of indicators from the original file with coefficent values above 0.1 is {CCA_coeff_anc.shape[0]}")
CCA_coeff_anc.to_csv(OUT+'/cca_antenatal_coverage_2016.csv', index=False, index_label=False)

,Indicators,CCA_coeff,CCA_coeff_abs
9,"Vitamin A coverage children 12-59 months, %",-0.203491,0.203491
34,Proportion of rural deaths,-0.105043,0.105043
36,Proportion of women population,0.118521,0.118521
37,Dependency ratio,-0.124174,0.124174
38,Sex ratio,-0.114493,0.114493


Number of indicators from the original file with coefficent values above 0.1 is 28


### Correlation analysis 2016

In [25]:
import scipy.stats  as stats
all(s_data2011.columns == s_data2016.columns)

True

In [26]:
corr2016 = s_data2016.corr()

In [27]:
antenatal_coverage_2016 = corr2016[['DHS: Proportion of antenatal coverage']]
antenatal_coverage_2016['abs_antenatal_coverage'] = np.absolute(antenatal_coverage_2016['DHS: Proportion of antenatal coverage'])
antenatal_coverage_2016 = antenatal_coverage_2016.sort_values(by='abs_antenatal_coverage', ascending=False)
antenatal_coverage_2016.drop('DHS: Proportion of antenatal coverage',axis=0, inplace=True)
antenatal_coverage_2016.reset_index(inplace=True)
corr_pvalues = []
for var in antenatal_coverage_2016['index']:
    pvalue = stats.pearsonr(s_data2016[var], s_data2016['DHS: Proportion of antenatal coverage'])[1]
    corr_pvalues.append(pvalue)
antenatal_coverage_2016['p_value'] = corr_pvalues
antenatal_coverage_2016 = antenatal_coverage_2016[antenatal_coverage_2016['p_value'] < 0.1]
antenatal_coverage_2016.sort_values(by = 'abs_antenatal_coverage', ascending=False)
display(antenatal_coverage_2016.head())
print(f"Number of indicators from the original file with correlation coefficient values above 0.1 is {antenatal_coverage_2016.shape[0]}")
antenatal_coverage_2016.to_csv(OUT+'/corr_antenatal_coverage_2016.csv', index=False, index_label=False)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/anaconda3/lib/python3.7/site-packages/scipy/stats/stats.py:3010: RuntimeWarning: invalid value encountered in double_scalars
  r = r_num / r_den


,index,DHS: Proportion of antenatal coverage,abs_antenatal_coverage,p_value
0,DHS: Proportion of attendant deliveries,0.774294,0.774294,6.170175e-14
1,DHS: Proportion of institutional deliveries,0.772737,0.772737,7.444530e-14
2,DHS: Proportion of C-sections,0.741550,0.741550,2.405134e-12
3,DHS: Dependency ratio,-0.406460,0.406460,8.599801e-04
4,DHS: Proportion of women (15-45 years),0.400571,0.400571,1.038380e-03


Number of indicators from the original file with correlation coefficient values above 0.1 is 17


### Combine results

In [28]:
antenatal_coverage_vars_2016 = list(set(CCA_coeff_anc['Indicators']).union(set(antenatal_coverage_2016['index'])))
len(list(set(CCA_coeff_anc['Indicators']).union(set(antenatal_coverage_2016['index']))))

35

## Intersect variables

In [29]:
print(len(antenatal_coverage_vars_2011))
print(len(antenatal_coverage_vars_2016))
antenatal_coverage_vars_union = list(set(antenatal_coverage_vars_2011).union(set(antenatal_coverage_vars_2016)))
antenatal_coverage_vars_inter = list(set(antenatal_coverage_vars_2011).intersection(set(antenatal_coverage_vars_2016)))
print(len(antenatal_coverage_vars_union))
print(len(antenatal_coverage_vars_inter))

69
35
86
18


## Performing HDBSCAN or Kmean clustering: Antenatal coverage

In [30]:
import os
import re
import glob
import conda
import hdbscan
import operator
import itertools
import numpy as np
import pandas as pd
import seaborn as sns
from config import Config
from collections import Counter
from matplotlib import pyplot as plt
conda_file_dir = conda.__file__
conda_dir = conda_file_dir.split('lib')[0]
proj_lib = os.path.join(os.path.join(conda_dir, 'share'), 'proj')
os.environ["PROJ_LIB"] = proj_lib
from mpl_toolkits.basemap import Basemap
from matplotlib.patches import Polygon
from matplotlib.lines import Line2D
from matplotlib.collections import PatchCollection
from sklearn.cluster import KMeans

In [31]:
def evaluate_hdbscan(input_df, min_samples, min_cluster_size, 
                     output, cluster_selection_method, 
                     fmin_samples, fmin_cluster_size,
                     prune=False, plot=True):
    samples = list(itertools.product(min_samples, min_cluster_size))
    counter = 0
    models = pd.DataFrame(columns=['min_samples',
                                   'min_cluster_size',
                                   'num_clusters_including_unclustered',
                                   'percent_of_unclustered_geos',
                                   'percent_of_maxclass',],index=range(len(samples)))

    df = input_df.copy(deep=True)
    for iteration in samples:
        model = hdbscan.HDBSCAN(min_samples=int(iteration[0]), 
                                min_cluster_size=int(iteration[1]), 
                                metric='euclidean', 
                                algorithm='best',
                                cluster_selection_method=cluster_selection_method, prediction_data=False).fit(df)
        models.loc[counter,'min_cluster_size'] = iteration[1]
        models.loc[counter, 'min_samples'] = iteration[0]
        models.loc[counter, 'num_clusters_including_unclustered'] = len(Counter(model.labels_))
        tmp_dict = dict(Counter(model.labels_))
        total = sum([v for k,v in tmp_dict.items()])
        tmp_dict = {k:round(v/total*100,2) for k,v in tmp_dict.items()}
        try:
            models.loc[counter, 'percent_of_unclustered_geos'] = tmp_dict.pop(-1)
        except KeyError as error:
            models.loc[counter, 'percent_of_unclustered_geos'] = 0 
        if len(tmp_dict) > 1:
            models.loc[counter, 'percent_of_maxclass'] = tmp_dict[max(tmp_dict.items(), key=operator.itemgetter(1))[0]]
        else:
            models.loc[counter, 'percent_of_maxclass'] = 100
        counter += 1
    if prune:
        out_model = hdbscan.HDBSCAN(min_samples=int(fmin_samples), 
                                min_cluster_size=int(fmin_cluster_size), 
                                metric='euclidean', 
                                algorithm='best',
                                cluster_selection_method=cluster_selection_method, prediction_data=False).fit(df)

    else:
        out_model = None

    if plot:
        plt.rcParams['figure.figsize'] = [20,10]
        plt.plot(models['num_clusters_including_unclustered'], label='Number of clusters including unclustered')
        plt.plot(models['percent_of_unclustered_geos'], label='Percent of unclustered geographies')
        plt.plot(models['percent_of_maxclass'], label='Size of larges cluster (%)')
        plt.xlabel("Iterations", fontsize=20)
        plt.ylabel("Value", fontsize=20)
        plt.savefig(os.path.split(output)[1] + "/finetune_parameteres.jpeg")
        plt.legend()
        plt.show()
    del(input_df, df)
    return models, out_model

In [32]:
OUT = '/Users/edinhamzic/Symphony/wb_bangladesh/Bangladesh/output/all/'

## Evaluate clustering method: HDBSCAN - leaf - 2011

In [33]:
tmp, out = evaluate_hdbscan(input_df=s_data2011[antenatal_coverage_vars_union], 
                       min_samples=Config.tune_min_sample, 
                       min_cluster_size=Config.tune_min_cluster,
                       output=OUT, cluster_selection_method ='leaf',
                       fmin_samples=3, fmin_cluster_size=12,
                       prune=True, plot=False)

In [34]:
tmp = tmp[tmp['num_clusters_including_unclustered'] <10]
tmp.sort_values('percent_of_unclustered_geos', ascending=True).head(10)

,min_samples,min_cluster_size,num_clusters_including_unclustered,percent_of_unclustered_geos,percent_of_maxclass
1,1,3,6,57.81,18.75
49,2,3,6,57.81,18.75
51,2,5,3,65.62,18.75
52,2,6,3,65.62,18.75
53,2,7,3,65.62,18.75
55,2,9,3,65.62,18.75
56,2,10,3,65.62,18.75
54,2,8,3,65.62,18.75
7,1,9,3,65.62,18.75
6,1,8,3,65.62,18.75


## Evaluate clustering method: HDBSCAN - eom - 2011

In [35]:
tmp, out = evaluate_hdbscan(input_df=s_data2011[antenatal_coverage_vars_union], 
                       min_samples=Config.tune_min_sample, 
                       min_cluster_size=Config.tune_min_cluster,
                       output=OUT, cluster_selection_method ='eom',
                       fmin_samples=3, fmin_cluster_size=12,
                       prune=True, plot=False)

In [36]:
tmp = tmp[tmp['num_clusters_including_unclustered'] <10]
tmp.sort_values('percent_of_unclustered_geos', ascending=True).head(10)

,min_samples,min_cluster_size,num_clusters_including_unclustered,percent_of_unclustered_geos,percent_of_maxclass
0,1,2,5,20.31,68.75
48,2,2,5,20.31,68.75
1,1,3,3,23.44,71.88
49,2,3,3,23.44,71.88
50,2,4,3,65.62,18.75
51,2,5,3,65.62,18.75
52,2,6,3,65.62,18.75
53,2,7,3,65.62,18.75
55,2,9,3,65.62,18.75
54,2,8,3,65.62,18.75


## Evaluate clustering method: HDBSCAN - leaf - 2016

In [37]:
tmp, out = evaluate_hdbscan(input_df=s_data2016[antenatal_coverage_vars_inter], 
                       min_samples=Config.tune_min_sample, 
                       min_cluster_size=Config.tune_min_cluster,
                       output=OUT, cluster_selection_method ='leaf',
                       fmin_samples=3, fmin_cluster_size=12,
                       prune=True, plot=False)

In [38]:
tmp = tmp[tmp['num_clusters_including_unclustered'] <10]
tmp.sort_values('percent_of_unclustered_geos', ascending=True).head(10)

,min_samples,min_cluster_size,num_clusters_including_unclustered,percent_of_unclustered_geos,percent_of_maxclass
1,1,3,4,70.31,18.75
2,1,4,3,75,18.75
48,2,2,5,85.94,4.69
96,3,2,3,85.94,10.94
276,6,38,1,100,100
295,7,9,1,100,100
294,7,8,1,100,100
293,7,7,1,100,100
292,7,6,1,100,100
291,7,5,1,100,100


## Evaluate clustering method: HDBSCAN - eom - 2016

In [39]:
tmp, out = evaluate_hdbscan(input_df=s_data2016[antenatal_coverage_vars_inter], 
                       min_samples=Config.tune_min_sample, 
                       min_cluster_size=Config.tune_min_cluster,
                       output=OUT, cluster_selection_method ='eom',
                       fmin_samples=3, fmin_cluster_size=12,
                       prune=True, plot=False)

In [40]:
tmp = tmp[tmp['num_clusters_including_unclustered'] <10]
tmp.sort_values('percent_of_unclustered_geos', ascending=True).head(10)

,min_samples,min_cluster_size,num_clusters_including_unclustered,percent_of_unclustered_geos,percent_of_maxclass
1,1,3,4,70.31,18.75
2,1,4,3,75,18.75
48,2,2,4,79.69,14.06
96,3,2,3,85.94,10.94
276,6,38,1,100,100
295,7,9,1,100,100
294,7,8,1,100,100
293,7,7,1,100,100
292,7,6,1,100,100
291,7,5,1,100,100


# Clustering with K-means

### Clustering Union List

In [41]:
kmeans_model = KMeans(n_clusters=4, random_state=0).fit(s_data2011[antenatal_coverage_vars_union])
predicted2016 = kmeans_model.predict(s_data2016[antenatal_coverage_vars_union])
d2011['cluster'] = kmeans_model.labels_
d2016['cluster'] = predicted2016

In [42]:
antenatal_coverage_vars_union.append('cluster')
d2011[antenatal_coverage_vars_union].to_csv(OUT+'/clusters_antenatal_coverage_union_2011.csv')
d2016[antenatal_coverage_vars_union].to_csv(OUT+'/clusters_antenatal_coverage_union_2016.csv')
print(d2011[antenatal_coverage_vars_union].shape)
print(d2016[antenatal_coverage_vars_union].shape)

(64, 87)
(64, 87)


In [43]:
print(Counter(d2011['cluster']))

Counter({3: 24, 1: 20, 0: 17, 2: 3})


In [44]:
print(Counter(d2016['cluster']))

Counter({3: 32, 1: 18, 0: 14})


### Clustering Intersection List

In [45]:
d2011 = d2011.drop('cluster', axis=1)
d2016 = d2016.drop('cluster', axis=1)

In [46]:
kmeans_model = KMeans(n_clusters=4, random_state=0).fit(s_data2011[antenatal_coverage_vars_inter])
predicted2016 = kmeans_model.predict(s_data2016[antenatal_coverage_vars_inter])
d2011['cluster'] = kmeans_model.labels_
d2016['cluster'] = predicted2016

In [47]:
print(Counter(d2011['cluster']))

Counter({2: 20, 3: 19, 1: 18, 0: 7})


In [48]:
print(Counter(d2016['cluster']))

Counter({1: 22, 3: 18, 2: 17, 0: 7})


In [49]:
antenatal_coverage_vars_inter.append('cluster')
d2011[antenatal_coverage_vars_inter].to_csv(OUT+'/clusters_antenatal_coverage_intersect_2011.csv')
d2016[antenatal_coverage_vars_inter].to_csv(OUT+'/clusters_antenatal_coverage_intersect_2016.csv')
print(d2011[antenatal_coverage_vars_inter].shape)
print(d2016[antenatal_coverage_vars_inter].shape)

(64, 19)
(64, 19)


In [50]:
tmp1 = d2011[antenatal_coverage_vars_inter]
tmp1 = tmp1.reset_index(col_level='DistricName')
tmp1['year'] = 2011
tmp2 = d2016[antenatal_coverage_vars_inter]
tmp2 = tmp2.reset_index(col_level='DistricName')
tmp2['year'] = 2016
tmp = pd.concat([tmp1, tmp2], axis=0)
print(tmp.shape)
tmp.to_csv(OUT+'clusters_antenatal_coverage_intersect_all.csv', index=False)

(128, 21)


In [51]:
tmp1 = d2011[antenatal_coverage_vars_union]
tmp1 = tmp1.reset_index(col_level='DistricName')
tmp1['year'] = 2011
tmp2 = d2016[antenatal_coverage_vars_union]
tmp2 = tmp2.reset_index(col_level='DistricName')
tmp2['year'] = 2016
tmp = pd.concat([tmp1, tmp2], axis=0)
print(tmp.shape)
tmp.to_csv(OUT+'clusters_antenatal_coverage_union_all.csv', index=False)

(128, 89)
